# Dashboard creation routines

In [8]:
from impact import Impact
import os
import json
import numpy as np

from pathlib import Path

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt

In [9]:
PREFIX = 'cu-inj-live-impact'

In [10]:
from PIL import Image, ImageOps

def fig2data ( fig ):
    """
    @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    @param fig a matplotlib figure
    @return a numpy 3D array of RGBA values
    """
    # draw the renderer
    fig.canvas.draw ( )
    # Get the RGBA buffer from the figure
    w,h = fig.canvas.get_width_height()
    buf = np.frombuffer ( fig.canvas.tostring_argb(), dtype=np.uint8 )
    buf.shape = ( w, h,4 )
    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll ( buf, 3, axis = 2 )
    return buf

def fig2img ( fig ):
    """
    @brief Convert a Matplotlib figure to a PIL Image in RGBA format and return it
    @param fig a matplotlib figure
    @return a Python Imaging Library ( PIL ) image
    """
    # put the figure pixmap into a numpy array
    buf = fig2data ( fig )
    w, h, d = buf.shape
    return Image.frombytes( "RGBA", ( w ,h ), buf.tobytes( ) )


In [11]:
def iscreen(impact_object, screen='OTR2', k1='x', k2='y', dpi=72):
    fig = impact_object.particles[screen].plot(k1, k2, return_figure=True, figsize=(5,4))
    fig.dpi=dpi
    fig.axes[2].set_title(screen)
    fig.tight_layout()
    return fig2img(fig)

In [23]:
def make_dashboard(impact_object=None, dat=None, itime=None, outpath='test/'):
    """
    Makes a composite dashboard image from data dict
    
    Returns the path to the figure written
    """
    if impact_object:
        I = impact_object   
    else:
        itime = dat['isotime']
        I = Impact.from_archive(dat['archive'])
        
    run_time = I.output['run_info']['run_time'] 
    # Main figure
    FIG0 = I.plot(['norm_emit_x','norm_emit_y'], 
       y2=['sigma_x', 'sigma_y', 'sigma_z'],
       
       ylim=(0,2e-6), figsize=(16,8), return_figure=True)
    
    title=f'{PREFIX}\n Acquired settings at {itime}, simulation run time: {run_time/60:5.1f} min'
    
    FIG0.tight_layout()
    FIG0.axes[0].set_title(title)
    
    DPI = 150
    FIG0.dpi=DPI
    im0 = fig2img(FIG0)
    
    # For short debugging runs
    if 'YAG02' not in I.particles:
        screen1='initial_particles'
        screen2='initial_particles'
        screen3='final_particles'
    else:
        screen1='YAG02'
        screen2='YAG03'
        screen3='OTR2'

    
    
    im1 = iscreen(I, screen=screen1, k1='x', k2='y', dpi=DPI)
    im2 = iscreen(I, screen=screen2, k1='x', k2='y', dpi=DPI)
    im3 = iscreen(I, screen=screen3, k1='x', k2='y', dpi=DPI)
    im4 = iscreen(I, screen=screen3, k1='px', k2='py', dpi=DPI)
    im5 = iscreen(I, screen=screen3, k1='delta_z', k2='delta_energy', dpi=DPI)
    
    SIZE =  (im0.width + im1.width, im1.height+im2.height+im3.height)
    ii = Image.new('RGB', SIZE)
    
    invim0 = ImageOps.invert(im0.convert('RGB'))
    ii.paste(invim0, (0, 10))
    
    ii.paste(im1, (300, im0.height))
    ii.paste(im2, (400+im1.width,im0.height))
    ii.paste(im3, (im0.width,0))
    ii.paste(im4, (im0.width,im3.height))
    ii.paste(im5, (im0.width,im4.height+im3.height))
    
    fname = f'{PREFIX}-{itime}-dashboard.png'
    fout = os.path.join(outpath, fname)
    
    ii.save(fout)
    
    return fout
    

In [24]:
##%%capture
#make_dashboard(dat=json.load(open('output/cu-inj-live-impact-2021-02-19T13:12:48-08:00.json')))

('test/cu-inj-live-impact-2021-02-19T13:12:48-08:00-dashboard.png',
 <PIL.Image.Image image mode=RGB size=3150x1800 at 0x2B620E92AA90>)